In [1]:
!pip uninstall qiskit
!pip uninstall qiskit-aer
!pip install qiskit=='0.7'
!pip install -U qiskit-aer
!pip install sympy=='1.7'
!pip uninstall qiskit-terra
!pip install qiskit-terra=='0.20.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.0/223.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.1 MB/s eta 0:00:00
  Created wheel for qiskit-aer: filename=qiskit_aer-0.1.0-py3-none-any.whl size=1289 sha256=62b0635268dfa19dac6415992a2ec17c746a3a0803aa24dbc3fd9efdc35c1645
  Stored in directory: /root/.cache/pip/wheels/10/96/28/f99aede11517f2e46cea9e5473d487e66582038f9696b228ea
  Created wheel for qiskit-terra: filename=qiskit_terra-0.7.2-cp310-cp310-linux_x86_64.whl size=744238 sha256=4af0510eab78839a4beba16011fb026f703fc82cc0bfd96200e9618239c28543
  Stored in directory: /root/.cache/pip/wheels/a7/cd/06/cd4022b3af93c8c342e04

In [4]:
from qiskit import QuantumCircuit, Aer
from qiskit.providers.aer import AerSimulator
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
import numpy as np
from random import choice, random

# Define measurement bases
bases = ['computational', 'hadamard']

def prepare_g_like_states(num_states):
    states = []
    for _ in range(num_states):
        qc = QuantumCircuit(3, 4)  # Three qubits for G-like state and four classical bits for measurement
        qc.h(0)
        qc.h(2)
        qc.x(1)
        qc.cx(0, 1)
        qc.cx(2, 1)
        states.append(qc)
    return states

def eavesdrop(qc, basis):
    if basis == 'hadamard':
        qc.h(0)
    qc.measure(0, 0)  # Measure the first qubit
    backend = AerSimulator()
    result = backend.run(qc, shots=1).result()
    counts = result.get_counts(qc)
    measurement = list(counts.keys())[0]
    qc = QuantumCircuit(3, 4)
    if measurement[0] == '1':
        qc.x(0)
    if basis == 'hadamard':
        qc.h(0)
    return qc, measurement

def bob_measures(qc, basis):
    if basis == 'hadamard':
        qc.h(0)
    qc.measure(0, 0)
    backend = AerSimulator()
    result = backend.run(qc, shots=1).result()
    counts = result.get_counts(qc)
    measurement = list(counts.keys())[0]
    return measurement

def third_party_measure(qc):
    qc.h(1)  # Apply Hadamard to Alice's qubit for the Bell measurement
    qc.cx(1, 2)  # Entangle Alice's qubit with Bob's qubit
    qc.measure([1, 2], [0, 1])  # Measure both qubits in Bell basis
    backend = AerSimulator()
    result = backend.run(qc, shots=1).result()
    counts = result.get_counts(qc)
    measurement = list(counts.keys())[0]
    return measurement

def quantum_fourier_transform(data):
    # Apply quantum Fourier transform
    qc = QuantumCircuit(len(data))
    for i in range(len(data)):
        qc.h(i)
        for j in range(i + 1, len(data)):
            qc.cu1(2 * np.pi / (2 ** (j - i)), j, i)
    return qc

def inverse_quantum_fourier_transform(data):
    # Apply inverse quantum Fourier transform
    qc = QuantumCircuit(len(data))
    for i in reversed(range(len(data))):
        for j in reversed(range(i)):
            qc.cu1(-2 * np.pi / (2 ** (i - j)), j, i)
        qc.h(i)
    return qc

def encode_data_as_qubits(data):
    # Encode classical data as qubits
    qc = QuantumCircuit(len(data))
    for i, bit in enumerate(data):
        if bit:
            qc.x(i)
    return qc

def decode_qubits_as_data(qubits):
    # Decode qubits as classical data
    backend = AerSimulator()
    result = backend.run(qubits, shots=1).result()
    counts = result.get_counts(qubits)
    measurement = list(counts.keys())[0]
    return int(measurement[::-1], 2)

def secure_multi_party_quantum_summation_protocol(data, parties, modulo, transmission_mode):
    if transmission_mode == 'complete-graph':
        # Every party prepares the initial quantum states and sends each of the other parties a sequence of prepared particles
        encoded_data = [encode_data_as_qubits(data) for _ in range(parties)]
        transformed_data = [quantum_fourier_transform(qc) for qc in encoded_data]
    elif transmission_mode == 'circle':
        # Every party prepares the initial quantum states and sends out one sequence of prepared particles
        encoded_data = encode_data_as_qubits(data)
        transformed_data = [quantum_fourier_transform(encoded_data) for _ in range(parties)]
    elif transmission_mode == 'tree':
        # Only one party prepares the initial quantum states and sends each of the other parties a sequence of prepared particles
        encoded_data = encode_data_as_qubits(data)
        transformed_data = [quantum_fourier_transform(encoded_data) for _ in range(parties - 1)]

    # Perform quantum Fourier transform and other necessary steps

    # Step 1: Perform quantum Fourier transform
    sum_result = sum(data) % modulo

    # Step 2: Implement calculation of addition in a secret-by-secret way
    # Not implemented in this simplified code

    # Step 3: Perform inverse quantum Fourier transform
    inverse_transformed_data = [inverse_quantum_fourier_transform(qc) for qc in transformed_data]

    # Step 4: Decode qubits as classical data
    decoded_results = [decode_qubits_as_data(qc) % modulo for qc in inverse_transformed_data]

    return decoded_results

# Main simulation
num_states = 4  # Number of G-like states to prepare

alice_states = prepare_g_like_states(num_states)
eve_does_eavesdrop = [random() < 0.5 for _ in range(num_states)]  # List of True/False indicating whether Eve eavesdrops on each state

chosen_alice_state_index = choice(range(num_states))
alice_qc = alice_states[chosen_alice_state_index]

alice_basis = choice(bases)  # Alice randomly selects a basis for the chosen state
eve_does_eavesdrop_state = eve_does_eavesdrop[chosen_alice_state_index]

if eve_does_eavesdrop_state:
    alice_qc, eve_measurement = eavesdrop(alice_qc, 'computational')  # Eve measures in Z basis

bob_basis = alice_basis  # Bob uses the same basis as Alice
bob_result = bob_measures(alice_qc, bob_basis)

if bob_result:  # Bob chooses to reflect the qubit
    third_party_measurement = third_party_measure(alice_qc)
    if third_party_measurement == '00':  # Bell state |Φ⁺⟩
        eve_detected = False
    else:  # Bell state |Φ⁻⟩
        eve_detected = True
else:
    eve_detected = False  # No reflection, so no chance to detect Eve

print(f"Alice chose state {chosen_alice_state_index + 1} from the prepared G-like states")
print(f"Alice prepared the G-like state using {alice_basis} basis")
print(f"Eve eavesdropped: {'Yes' if eve_does_eavesdrop_state else 'No'}{' measuring in Z basis' if eve_does_eavesdrop_state else ''}")
if eve_does_eavesdrop_state:
   print(f"Eve's measurement result: {eve_measurement}")
if bob_result == '1':
    print(f"Bob reflected the qubit and performed Bell measurement")
    if eve_detected:
        print("Eve detected!")
    else:
        print("Eve not detected.")
print(f"Bob's measurement result: {bob_result} using {bob_basis} basis")
print("Discrepancy introduced: " + ("Yes" if bob_result != '0' else "No"))

# Perform secure multi-party quantum summation protocol
data = [3, 5, 7]
parties = 3
modulo = 10
transmission_mode = 'complete-graph'  # Choose transmission mode: 'complete-graph', 'circle', or 'tree'
result = secure_multi_party_quantum_summation_protocol(data, parties, modulo, transmission_mode)
print("Secure Multi-Party Quantum Summation Result:", result)


ModuleNotFoundError: No module named 'qiskit.providers.aer'

In [11]:
from qiskit import QuantumCircuit
import numpy as np
from random import choice, random

# Define measurement bases
bases = ['computational', 'hadamard']

def prepare_g_like_states(num_states):
    states = []
    for _ in range(num_states):
        qc = QuantumCircuit(3, 4)  # Three qubits for G-like state and four classical bits for measurement
        qc.h(0)
        qc.h(2)
        qc.x(1)
        qc.cx(0, 1)
        qc.cx(2, 1)
        states.append(qc)
    return states

def eavesdrop(qc, basis):
    if basis == 'hadamard':
        qc.h(0)
    qc.measure(0, 0)  # Measure the first qubit
    # Simulate eavesdropping (omitted in this simplified version)
    qc = QuantumCircuit(3, 4)
    # Simulated eavesdropping, not included in this version
    return qc, '0'  # Simulated measurement result

def bob_measures(qc, basis):
    if basis == 'hadamard':
        qc.h(0)
    qc.measure(0, 0)
    # Simulate Bob's measurement (omitted in this simplified version)
    return '0'  # Simulated measurement result

def third_party_measure(qc):
    qc.h(1)  # Apply Hadamard to Alice's qubit for the Bell measurement
    qc.cx(1, 2)  # Entangle Alice's qubit with Bob's qubit
    qc.measure([1, 2], [0, 1])  # Measure both qubits in Bell basis
    # Simulate third-party measurement (omitted in this simplified version)
    return '00'  # Simulated measurement result

def secure_multi_party_quantum_summation_protocol(data, parties, modulo, transmission_mode):
    # Simulation of secure multi-party quantum summation protocol (omitted in this simplified version)
    return [sum(data) % modulo] * parties  # Simulated result

# Main simulation
num_states = 4  # Number of G-like states to prepare

alice_states = prepare_g_like_states(num_states)
eve_does_eavesdrop = [random() < 0.5 for _ in range(num_states)]  # List of True/False indicating whether Eve eavesdrops on each state

chosen_alice_state_index = choice(range(num_states))
alice_qc = alice_states[chosen_alice_state_index]

alice_basis = choice(bases)  # Alice randomly selects a basis for the chosen state
eve_does_eavesdrop_state = eve_does_eavesdrop[chosen_alice_state_index]

if eve_does_eavesdrop_state:
    alice_qc, eve_measurement = eavesdrop(alice_qc, 'computational')  # Eve measures in Z basis

bob_basis = alice_basis  # Bob uses the same basis as Alice
bob_result = bob_measures(alice_qc, bob_basis)

if bob_result:  # Bob chooses to reflect the qubit
    third_party_measurement = third_party_measure(alice_qc)
    if third_party_measurement == '00':  # Bell state |Φ⁺⟩
        eve_detected = False
    else:  # Bell state |Φ⁻⟩
        eve_detected = True
else:
    eve_detected = False  # No reflection, so no chance to detect Eve

print(f"Alice chose state {chosen_alice_state_index + 1} from the prepared G-like states")
print(f"Alice prepared the G-like state using {alice_basis} basis")
print(f"Eve eavesdropped: {'Yes' if eve_does_eavesdrop_state else 'No'}{' measuring in Z basis' if eve_does_eavesdrop_state else ''}")
if eve_does_eavesdrop_state:
   print(f"Eve's measurement result: {eve_measurement}")
if bob_result == '1':
    print(f"Bob reflected the qubit and performed Bell measurement")
    if eve_detected:
        print("Eve detected!")
    else:
        print("Eve not detected.")
print(f"Bob's measurement result: {bob_result} using {bob_basis} basis")
print("Discrepancy introduced: " + ("Yes" if bob_result != '0' else "No"))

# Perform secure multi-party quantum summation protocol
data = [15, 5, 7]
parties = 3
modulo = 10
transmission_mode = 'complete-graph'  # Choose transmission mode: 'complete-graph', 'circle', or 'tree'
result = secure_multi_party_quantum_summation_protocol(data, parties, modulo, transmission_mode)
print("Secure Multi-Party Quantum Summation Result:", result)


Alice chose state 2 from the prepared G-like states
Alice prepared the G-like state using computational basis
Eve eavesdropped: Yes measuring in Z basis
Eve's measurement result: 0
Bob's measurement result: 0 using computational basis
Discrepancy introduced: No
Secure Multi-Party Quantum Summation Result: [7, 7, 7]
